In [1]:
import torch as tc
import pandas as pd
import numpy as np
import open3d as otd
from tqdm import tqdm
import matplotlib.pyplot as plt
otd_vector3d = otd.utility.Vector3dVector
from src.star.star import STAR
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
from src.mesh_manipulation import save_obj
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
their_semantics = [
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Height (m)',
]
our_semantic = [
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'stature', # 5.1.1
]

curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

In [3]:
subdivided_bodies = tc.load('data/subdivided_bodies.pt')
measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures.index = measures['Subject']
measures = measures[measures['Measuring station'] == "MOVE4D"]

mfd_gender_measures = dict()
for gender in genders:
    mfd_gender_measures[gender] = measures[measures['Sex'] == gender]
    mfd_gender_measures[gender] = mfd_gender_measures[gender][their_semantics]
    mfd_gender_measures[gender].columns = our_semantic
    mfd_gender_measures[gender]['stature'] *= 1000

In [4]:
selected_subjects = dict()
selected_measures = dict()

for gender in genders:
    gender_measures = measures[measures['Sex'] == gender]
    temp_measures = gender_measures[their_semantics].iloc[::2]
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [14]:
body = subdivided_bodies['vertices']['female'][selected_subjects['female']]
template = subdivided_bodies['faces']['female']+1
save_obj(path="output/female_body.obj", pontos=body, faces=template)

tensor(0)


In [5]:
gender_curves = dict()
for gender in genders:
    print(f'SEGMENTING {gender.upper()} BODIES', end='')
    faces = subdivided_bodies['faces'][gender].to(device)
    bodies = subdivided_bodies['vertices'][gender]
    body = bodies[selected_subjects[gender]].to(device)
    measures = selected_measures[gender]
    result = CurveGenerator.get_curves(body, measures, faces, device, gender)
    gender_curves[gender] = result[0]
tc.save(gender_curves, "data/gender_curves.zip")

SEGMENTING FEMALE BODIES

processing body: 100%|██████████| 44/44 [00:21<00:00,  2.06it/s]


SEGMENTING MALE BODIES

processing body: 100%|██████████| 44/44 [00:23<00:00,  1.84it/s]


In [9]:
gender_measures = dict()
for gender in genders:
    print(f'MEASURING {gender.upper()} BODIES', end='')
    gender_measures[gender] = [[]]*5
    for segments_index, curves_segments in enumerate(gender_curves[gender]):
        gender_measures[gender][segments_index] = []
        for curves_index, curves in enumerate(tqdm(curves_segments)): # bust, torso, leg, arm, neck

            gender_measures[gender][segments_index].append([])
            for body in subdivided_bodies['vertices'][gender]:
                position = CurveUtils.generate_positions(tc.FloatTensor(curves).to(device), body.to(device))
                calculated_measures = CurveUtils.calculate_distances(position)
                gender_measures[gender][segments_index][curves_index].append(calculated_measures)

            calculated_measures = gender_measures[gender][segments_index][curves_index]
            gender_measures[gender][segments_index][curves_index] = tc.FloatTensor(calculated_measures)
        gender_measures[gender][segments_index] = tc.row_stack(gender_measures[gender][segments_index])

tc.save(gender_measures, "data/calculated_measures.zip")


MEASURING FEMALE BODIES

100%|██████████| 6080/6080 [02:24<00:00, 42.09it/s]


MEASURING MALE BODIES

100%|██████████| 6800/6800 [02:43<00:00, 41.53it/s]


In [10]:
gender_calculated_measures = tc.load("data/calculated_measures.zip")

In [11]:
gender_measures_wc = dict()
for gender in genders:
    gender_measures_wc[gender] = []
    for index in range(0,5):
        gender_measures_wc[gender].append(gender_measures[gender][index])
        if index == 1:
            gender_measures_wc[gender].append(gender_measures[gender][index])
gender_measures = gender_measures_wc

In [18]:
best_gender_measures = dict() ## caso minimo

for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-1]):
        measured = gender_measures[gender][index].T[::2].T
        ground_truth = mfd_gender_measures[gender][curve][::2]
        result = (measured - tc.FloatTensor(ground_truth/10).unsqueeze(0)).abs()
        min_rows_values, min_rows_indices = result.min(0)
        min_columns_values, min_columns_indices = min_rows_values.min(0)
        best = min_rows_indices[min_columns_indices]
        best_gender_measures[gender].append((
            best.numpy(),
            result[best].min().numpy(),
            result[best].max().numpy(),
            result[best].mean().numpy(),
            result[best].std().numpy()
        ))
gender_results = {
    'male': pd.DataFrame(best_gender_measures['male'], columns=['best', 'min', 'max', 'mean', 'std']),
    'female': pd.DataFrame(best_gender_measures['female'], columns=['best', 'min', 'max', 'mean', 'std'])
}
print("male errors:")
print(gender_results['male'])
print("\nfemale errors:")
print(gender_results['female'])

male errors:
   best           min        max        mean         std
0   181  0.0070648193  6.1884003   2.5387866   1.3964223
1   339  0.0016403198  1.5482559  0.51437485  0.41830182
2    72  0.0002822876   5.557579   1.8663875   1.1089624
3   541  0.0011978149  2.6108322   0.8296479  0.65260595
4   392  0.0019607544  1.3141403  0.41693634  0.32365268
5  5641  8.773804e-05  2.8908653  0.80076283   0.5849153

female errors:
   best            min        max        mean         std
0   161   0.0009918213  6.2503586   3.0942974   1.6616387
1   271    0.002204895  7.9995575    3.807967   2.2380903
2    79  0.00037384033   8.037308    1.816402   1.9512606
3   475  0.00024795532  3.0649414   0.8721026   0.7574359
4   326    0.000705719  3.6671715   1.6733981  0.98030597
5  2331   2.670288e-05  2.2580605  0.76263106  0.66059804


In [21]:
best_gender_measures = dict() ## caso médio
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-1]):
        measured = gender_measures[gender][index].T[::2].T
        ground_truth = mfd_gender_measures[gender][curve][::2]
        result = (measured - tc.FloatTensor(ground_truth/10).unsqueeze(0)).abs()
        min_rows_values = result.mean(1)
        min_columns_values, min_columns_indices = min_rows_values.min(0)
        best = min_columns_indices
        best_gender_measures[gender].append((
            result[best].min().numpy(),
            result[best].max().numpy(),
            result[best].mean().numpy(),
            result[best].std().numpy()
        ))
gender_results = {
    'male': pd.DataFrame(best_gender_measures['male'], columns=['min', 'max', 'mean', 'std'], index=our_semantic[:-1]),
    'female': pd.DataFrame(best_gender_measures['female'], columns=['min', 'max', 'mean', 'std'], index=our_semantic[:-1])
}
print("male errors:")
print(gender_results['male'])
print("\nfemale errors:")
print(gender_results['female'])

male errors:
                           min         max        mean         std
bust_chest_girth    0.03806305    5.150383    1.197003   1.1326014
waist_girth       0.0016403198   1.5482559  0.51437485  0.41830182
hip_girth          0.012916565   1.4537582  0.48320812   0.3904905
thigh_girth        0.022727966    2.086773    0.673372  0.52441585
upper_arm_girth    0.017017365  0.88785934   0.3111993  0.21203174
neck_girth         0.030872345   1.4503517  0.45744452  0.40859184

female errors:
                            min        max        mean         std
bust_chest_girth   0.0132369995  3.8647156   1.6071029   1.1682594
waist_girth         0.033843994   3.912056   1.1335517   1.0194021
hip_girth            0.06463623  4.7434235   1.2451998   1.0968579
thigh_girth       0.00024795532  3.0649414   0.8721026   0.7574359
upper_arm_girth     0.012990952  1.4030724   0.5278891   0.4078794
neck_girth          0.044963837  1.2938881  0.44332218  0.32921478


In [31]:
(gender_results['male']*10).astype(float).round(decimals=2) #5,14 / 5,11 6,49/6,05

,min,max,mean,std
bust_chest_girth,0.38,51.50,11.97,11.33
waist_girth,0.02,15.48,5.14,4.18
hip_girth,0.13,14.54,4.83,3.90
thigh_girth,0.23,20.87,6.73,5.24
upper_arm_girth,0.17,8.88,3.11,2.12
neck_girth,0.31,14.50,4.57,4.09


In [29]:
(gender_results['female']*10).astype(float).round(decimals=2)['mean'].mean() # 7,91 / 7,96

9.715

In [17]:
best_gender_curves = dict()
for gender in genders:
    all_positions = []
    best_gender_curves[gender] = []
    for index, curve in enumerate(our_semantic[:-1]):
        best = gender_results[gender].loc[index]['best']
        coordinates = gender_curves[gender][curve_index[curve]][best]
        best_gender_curves[gender].append(coordinates)
        faces = subdivided_bodies['faces'][gender].to(device)
        bodies = subdivided_bodies['vertices'][gender]
        body = bodies[selected_subjects[gender]].to(device)
        position = CurveUtils.generate_positions(coordinates, body.to(device))
        all_positions.append(position)
    save_obj(f'output/{gender}_points.obj', tc.row_stack(all_positions))
tc.save(best_gender_curves, 'data/selected_gender_curves.zip')

KeyError: 0

In [ ]:
best_gender_measures = dict()
for idx, gender in enumerate(genders):
    best_gender_measures[gender] = []
    for index, curve in enumerate(our_semantic[:-1]):
        best = gender_results[gender].loc[index]['best']
        measured = gender_measures[gender][index][best].numpy()
        best_gender_measures[gender].append(measured*10)
    best_gender_measures[gender].append(tc.arange(2).repeat(36).numpy()+1)
    best_gender_measures[gender].append([gender]*72)
    best_gender_measures[gender].append(["our"]*72)
    best_gender_measures[gender].append(mfd_gender_measures[gender].index)

In [ ]:
aditional_semantic = ['repetition', "gender", "measures_station", 'subject']
our_measures = pd.concat([
    pd.DataFrame(best_gender_measures['female'], index=our_semantic[:-1]+aditional_semantic).T,
    pd.DataFrame(best_gender_measures['male'], index=our_semantic[:-1]+aditional_semantic).T
])
our_measures.to_pickle("data/our_measures.zip")